# 01. Revisit Deep Neural Networks

### Machine Learning Problem

![img1](./img/img1.png)

* 전통적인 ML에서는 모델 종류와 Loss function에 대해 정의해줘야 한다.
* 모델을 정하면 그 모델을 결정지을 parameter를 추정해야하는데 이 parameter를 학습하는 과정이 보통의 학습과정이다.
* 이 때 parameter를 theta라 하면 모델의 출력값과 실제값의 차이가 얼마나 다르냐를 정의한 것이 Loss function. 
* Loss function이 최소가 되는 theta값을 찾아가는 것이 학습의 과정이다.
* ***고정된 입력값에 대해서는 고정된 출력값이 나온다***

#### Define Function

![img2](./img/img2.png)

* 여기서 DNN은 모델의 종류를 의미한다. f에 해당.
* DNN에서 Loss function은 MSE(Mean Squared Error) or Crossentropy 만 사용한다. WHY??(거의)
* 대부분의 DNN이 Backpropagation(이하에서 Backpropa) 을 통해 학습이 되는데 Backpropa는 2가지 가정을 가지고 있고 이 가정을 만족하는 Loss    Function 만 사용할 수 있기 때문이다. 가정은 다음과 같다.
* ![img3](./img/img3.png)
    1. Total loss of DNN over training samples is the sum of loss for each training sample.**(training 샘플 전체의 Loss값 = 각각 샘플의 Loss의 합)**
    1. Loss for each training example is a function of final output of DNN.(Loss function의 입력인자는 **네트워크 종단의 출력값과 정답값으로만 구성.** 즉, 네트워크 중간 레이어에서 나온 출력값은 사용하지 않는다. Googlenet의 경우 중간값을 사용하는 것처럼 보이지만 종단이라고 생각하고 뒤로 Backpropa 한다.)

#### Training/Learning

- Training데이터 전체에 대한 Loss를 최소화하는 theta를 찾는데 대부분 Gradient Descent 를 사용한다.
- G.D는 Iterative Method. 점차적으로 solution을 찾는 방법. 여기에는 2가지 정의가 필요하다.
    1. theta를 어떻게 update 할 것인가?
        - Loss가 줄어들기만 하면 theta를 update
    1. 언제 update를 멈출 것인가?
        - theta를 갱신해도 Loss가 줄어들지 않으면 stop

- But, 우리가 다루는 DNN에서 theta의 dimension은 매우 크다. 그렇다면 **어떻게 theta를 바꿔야 loss가 줄어드는 쪽으로 이동할까?**
- ![img4](./img/img4.png)
- Taylor expansion의 근삿값으로 1차미분 term까지만 사용한다. 더 많은 차수의 미분 term을 사용할수록 더 넓은 범위에서 오차가 줄어든다.
- 1차 미분 term까지만 사용하는 의미 : 그 데이터 포인트 근방에서는 오차가 거의 없다. 거의 같은 값으로 보인다.
- Approximation에서 L(theta)를 좌변으로 이항하면 그 좌변(delta L)이 음수가 돼야 loss가 줄어드는것이다. 따라서 Gradient(L)*delta(theta)를 음수로 만들어야 한다. How??
    - delta(theta)가 -Gradient(L) 이면 delta(L)은 항상 음수가 된다. 위 식에서 eta는 learning rate로 항상 양수인 수다.
    - 여기서 learning rate를 항상 작은 값(0.001, 0.01 etc.)을 사용하는 이유를 알 수 있다. 
        - loss가 감소하려면 delta(theta)=-Gradient 여야 한다. 그러면 어떤 Gradient 값을 쓸 것인가? Approximation에서 1차미분값까지만 사용했으므로 **데이터 포인트 근방의 Gradient값**이어야 감소 방향의 오차가 줄어든다.

- ![img5](./img/img5.png)
- Line 1 은 Backpropa의 가정이므로 성립한다고 치면 그것의 Gradient값에 대해서도 성립
- 최적화 문제에서 상수배하거나 상수 term을 더하는 것은 영향을 주지 않음(training 중간에 데이터 개수가 변하지 않는다는 가정하에). 따라서 우변을 데이터 개수로 나누면 Line 3에서 우변은 sample별 평균 Gradient 를 의미.
- But, 데이터 개수가 너무 크기 때문에 batch를 사용한다. 전체 training데이터셋의 평균 Gradient 와 training셋에서 M개를 임의추출하여 구한 M개의 평균 Gradient 값이 같을 것이라고 기대. -> Stochastic G.D
- 이렇게 구한 Gradient를 가지고 theta(weight, bias)를 갱신한다.
- 정리하면 **원래 전체 training DB에 대한 Gradient를 구해야 하지만 연산량이 너무 많기 때문에 평균 Gradient를 사용하고 여기서 또 전체에 대한 평균이 아니라 batch의 평균을 사용한다.**

- ![img6](./img/img6.png)
- output layer에서 Error signal은 네트워크 출력값에 대한 Loss function의 미분값에 Activation function의 미분값(activate function 입력값에 대한)을 곱한 값이다. 여기서 곱은 element-wise 곱.
- bias에 대한 Gradient 값은 Error signal
- weight에 대한 Gradient 값은 Error signal*해당layer입력값
- ref) http://neuralnetworksanddeeplearning.com/chap2.html 

#### Loss function viewpoint

- Loss function중에서 MSE와 Cross Entropy 중 어떤 것이 더 좋은가?? 
- 두 가지 관점에서 볼 수 있다. 
    1. Backpropagation 알고리즘이 잘 동작하는 관점 : Cross Entropy가 더 좋다.
    1. **Maximum likilihood 관점 : 출력값이 continuous value 이면 MSE, discrete value이면 Cross Entropy가 좋다.**
- 두 가지 관점에서 봤을때 이러한 해석이 있다는 것이지 맞다는 것은 아니다.

### Loss function viewpoint 1 : Backpropagation

- 아주 간단한 NN모델을 예시로 보자.
- ![img7](./img/img7.png)
- 위 모델을 학습하는 과정에서 loss function이 MSE이면 W,b의 초기값이 학습에 많은 영향을 끼친다. WHY?
    - Error Signal 수식을 보면 activation function 미분 term이 살아있는데 sigmoid 함수의 미분값은 커봤자 약 0.25정도이다.
    - 위의 경우 activation function이 sigmoid 함수인데 초기값에 따라서 Gradient값이 차이가 있다. 초기값이 sigmoid의 미분값이 거의 0인 지점이라면 Gradient값이 거의 0이므로 학습이 매우 느리게 될 수 있다. 
    - 이 문제가 레이어가 깊을수록 레이어를 지나면서 Error Signal이 0.25배로 계속 줄어들다가 나중에는 거의 0이 되어버려서 학습이 안되게 된다.(Vanishing Gradient Problem)
     

![img9](./img/img9.png)
- 반면 Cross Entropy함수일때 Error Signal을 구해보면 위 표처럼 절묘하게 activation function 미분 term이 사라지게 된다.
- 이에 따라서 MSE와 비교했을때 상대적으로 Vanishing Gradient Problem에서 자유롭다.
![img10](./img/img10.png)
- loss function에 따라서 학습되는 정도에 차이를 보이는 것을 그래프를 통해 알 수 있다.
- 확실히 Cross Entropy 일때는 초기값에 영향을 덜 받는다.

### Loss function viewpoint 2 : Maximum likelihood

### Maximum likelihood for AE